<a href="https://colab.research.google.com/github/masoud-moghini/PyTorch-Computer-Vision-Cookbook/blob/master/chapter9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from pathlib import Path
import cv2
from random import randint
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
tfms = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

In [43]:
class SegData(Dataset):
  def __init__(self, split):
    self.items= Path(f'/content/drive/MyDrive/semanticsegmentation/{split}').stem
    self.split = split

  def __len__(self):
    return len(self.items)

  def __getitem__(self, idx):
    img = cv2.imread(f'/content/drive/MyDrive/semanticsegmentation/{self.split}/{self.items[idx]}.png')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(224,224))
    mask = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    return img,mask
  def choose(self): return self[randint(len(self))]
  def collate_fn(self,batch):
    imgs,masks = list(*zip(batch))
    imgs = torch.cat([tfms(img.copy()/255)[None] for img in imgs])
    masks = torch.cat([tfms(mask.copy()/255)[None] for mask in masks])
    return imgs,masks

In [45]:
trn_ds = SegData('train')
val_ds = SegData('test')
trn_dl = DataLoader(trn_ds, batch_size=4, shuffle=True,collate_fn=trn_ds.collate_fn)
val_dl = DataLoader(val_ds, batch_size=1, shuffle=True,collate_fn=val_ds.collate_fn)

In [35]:
fo.shape

(4, 2, 3)

In [38]:
lo = tfms(fo.copy()/255)

In [37]:
lo.shape

torch.Size([3, 4, 2])